# Czynniki ryzyka chorób sercowo-naczyniowych:
# analiza danych demograficznych i zdrowotnych

### Wstęp:
#### Choroba serca jest złożonym procesem, na który wpływ ma wiele czynników zdrowotnych oraz demograficznych. Podczas analizy poniższego zbioru danych skupię się na kluczowych determinantach ryzyka oraz relacji między nimi. Może to pomóc lepiej zrozumieć, jak poszczególne czynniki mogą mieć znaczenie dla zdrowia serca i jakie wnioski można wyciągnąć, aby promować zdrowszy styl życia.

### Cel analizy:
#### Badanie czynników ryzyka chorób serca w kontekście demografii i nawyków zdrowotnych


### Źródło danych: 
#### [https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease](url)

### Opis danych:

#### Dane pochodzą z amerykańskiego systemu Behavioral Risk Factor Surveillance System (BRFSS), który jest największym globalnym programem ankiet telefonicznych badających stan zdrowia populacji. Zostały one zebrane i udostępnione przez Centers for Disease Control and Prevention (CDC) jako część corocznego monitorowania zdrowia mieszkańców Stanów Zjednoczonych. Analizowany zbiór danych obejmuje rok 2020.
#### Analizowany zbiór danych zawiera zmienne dotyczące stylu życia, nawyków zdrowotnych i cech demograficznych, które pozwalają na zbadanie ryzyka chorób serca

#### Opis wszystkich kolumn ze zbioru danych (pogrubione - wartości użyte w analizie):

- **HeartDisease - Czy u badanej osoby zdiagnozowano chorobę serca? (Yes / No)**
- **BMI	- Wskaźnik masy ciała (Body Mass Index)**
- **Smoking	- Czy badana osoba jest osobą palącą? (Yes / No)**
- AlcoholDrinking	- Czy u badanej osoby występuje nadmierne spożycie alkoholu? (Yes / No) (mężczyźni > 5 drinków miesięcznie, kobiety 4)
- Stroke - Czy u badanej osoby wystąpił udar? (Yes / No)
- PhysicalHealth	- Liczba dni w ciągu ostatniego miesiąca, w których występowały problemy zdrowotne fizyczne (np. ból, choroba).
- MentalHealth	- Liczba dni w ciągu ostatniego miesiąca, w których występowały problemy zdrowotne psychiczne (np. depresja, stres, lęki).
- DiffWalking	- Czy u badanej osoby występują trudności z chodzeniem lub poruszaniem się? (Yes / No)
- **Sex	- Płeć badanej osoby (Male / Female).**
- **AgeCategory	- Kategoria wiekowa (np. 18-24, 25-29, ..., 80 or older).**
- Race	- Rasa lub grupa etniczna (White, Black, Asian, American Indian/Alaskan Native, Hispanic, Other).
- Diabetic	- Czy u badanej osoby zdiagnozowano cukrzycę? (Yes / No)
- PhysicalActivity	- Czy u badanej osoby występuje regularna aktywność fizyczna? (Yes / No)
- GenHealth	- Samoocena ogólnego stanu zdrowia (Excellent, Very good, Good, Fair, Poor).
- SleepTime	- Średnia liczba godzin snu na dobę.
- Asthma	- Czy u badanej osoby zdiagnozowano astmę? (Yes / No)
- KidneyDisease	- Czy u badanej osoby przewlekłą chorobę nerek? (Yes / No)
- SkinCancer	- Czy u badanej osoby zdiagnozowano raka skóry? (Yes / No)

### Pobranie i załadowanie danych

In [12]:
%pip install kagglehub==0.3.6

dbutils.library.restartPython()

Note: you may need to restart the kernel to use updated packages.


NameError: name 'dbutils' is not defined

In [ ]:
from pyspark.sql.functions import *

In [ ]:
import kagglehub
import os


path = kagglehub.dataset_download("kamilpytlak/personal-key-indicators-of-heart-disease")

csv_path_local = f"{path}/2020/heart_2020_cleaned.csv"


if not os.path.exists(csv_path_local):
    raise FileNotFoundError(f"Plik CSV nie istnieje w lokalizacji: {csv_path_local}")


dbfs_csv_path = "dbfs:/FileStore/tables/heart_2020_cleaned.csv"
dbutils.fs.cp(f"file:{csv_path_local}", dbfs_csv_path)
print(f"Plik został skopiowany do DBFS: {dbfs_csv_path}")


df = spark.read.csv(dbfs_csv_path, header=True, inferSchema=True)


df.display(5)
df.printSchema()


### Wstępne czyszczenie danych

#### 1) brak wartości NULL w zbiorze danych.

#### 2) usunięcie nierealnej wartości SleepTime = 24h


In [16]:
df = df.filter(df["SleepTime"] != 24)

df.display()

NameError: name 'df' is not defined

###  Zapisanie danych do Hive

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS heart_disease_db")
spark.sql("USE heart_disease_db")


df.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("heart_disease_data")


In [ ]:
%sql
DESCRIBE TABLE heart_disease_data;

col_name,data_type,comment
HeartDisease,string,null
BMI,double,null
Smoking,string,null
AlcoholDrinking,string,null
Stroke,string,null
PhysicalHealth,double,null
MentalHealth,double,null
DiffWalking,string,null
Sex,string,null
AgeCategory,string,null


### Agregacje i wizualizacje

#### 1) Wykres kołowy: Podział badanej populacji ze względu na płeć

In [ ]:
%sql
SELECT 
    Sex,
    COUNT(*) AS Count,
    ROUND((COUNT(*) * 100.0) / SUM(COUNT(*)) OVER (), 2) AS Percentage
FROM heart_disease_data
GROUP BY Sex
ORDER BY Percentage DESC;

Sex,Count,Percentage
Female,167789,52.47
Male,151976,47.53


Databricks visualization. Run in Databricks to view.

##### Wnioski:
- #####  Różnica między płciami wynosi tylko 5%, co oznacza, że próbka jest dobrze zbalansowana i reprezentatywna.

#### 2) Wykres kołowy: Podział badanej populacji z chorobami serca ze względu na płeć

In [ ]:
%sql
SELECT 
    Sex,
    COUNT(*) AS Count,
    ROUND((COUNT(*) * 100.0) / SUM(COUNT(*)) OVER (), 2) AS Percentage
FROM heart_disease_data
WHERE HeartDisease = 'Yes'
GROUP BY Sex
ORDER BY Percentage DESC;


Sex,Count,Percentage
Male,16137,58.97
Female,11230,41.03


Databricks visualization. Run in Databricks to view.

##### Wnioski:
- ##### 59% osób z chorobami serca to mężczyźni, a 41% to kobiety.
- ##### mężczyźni chorują częściej niż wynikałoby to tylko z ich udziału w populacji.

#### 3) Wykres kołowy: Porównanie liczby osób z chorobami serca i bez nich

In [ ]:
heart_disease_counts = df.groupBy("HeartDisease").agg(count("*").alias("Count"))

display(heart_disease_counts)


HeartDisease,Count
No,292398
Yes,27367


Databricks visualization. Run in Databricks to view.

##### Wnioski:
- #####  Lekko ponad 8,50% badanej populacji ma zdiagnozowaną chorobę serca, gdzie prawie 91.5% takiej nie posiada.

#### 4)  wykres liniowy: Liczba obserwacji występowania chorób serca w badanej populacji w rożnych grupach wiekowych z podziałem na płeć

In [ ]:
heart_disease_data = df.filter(df["HeartDisease"] == "Yes")
age_sex_counts = heart_disease_data.groupBy("AgeCategory", "Sex").agg(count("*").alias("Count"))

display(age_sex_counts)

AgeCategory,Sex,Count
18-24,Male,79
70-74,Male,3037
75-79,Female,1642
80 or older,Female,2627
55-59,Female,908
70-74,Female,1810
30-34,Female,99
45-49,Female,320
35-39,Male,148
40-44,Female,243


Databricks visualization. Run in Databricks to view.

##### Wnioski:
- ##### Mężczyźni częściej chorują na choroby serca niż kobiety w każdej grupie wiekowej.
- ##### Różnica ta pogłębia się po 50 roku życia, kiedy liczba przypadków wśród mężczyzn rośnie znacznie szybciej niż u kobiet.
- ##### W grupie wiekowej 75-79 lat widoczny jest spadek liczby przypadków u kobiet, jednak po 80 roku życia ponownie wzrasta, co może sugerować, że kobiety żyją dłużej, ale w starszym wieku ich ryzyko zachorowania również rośnie.

#### 5) Wykres słupkowy: Rozkład procentowy różnych kategorii BMI wśród osób z chorobą serca

In [ ]:
heart_disease_df = df.filter(col("HeartDisease") == "Yes")


heart_disease_df = heart_disease_df.withColumn(
    "BMI_Category",
    when(col("BMI") < 18.5, "Niedowaga")
    .when((col("BMI") >= 18.5) & (col("BMI") < 25), "Waga prawidłowa")
    .when((col("BMI") >= 25) & (col("BMI") < 30), "Nadwaga")
    .when((col("BMI") >= 30) & (col("BMI") < 35), "Otyłość I stopnia")
    .when((col("BMI") >= 35) & (col("BMI") < 40), "Otyłość II stopnia")
    .otherwise("Otyłość III stopnia"))


bmi_distribution = heart_disease_df.groupBy("BMI_Category").agg(count("*").alias("Count"))


total_count = heart_disease_df.count()
bmi_distribution = bmi_distribution.withColumn("Percentage", round((col("Count") / total_count) * 100, 2))


bmi_distribution = bmi_distribution.withColumn(
    "BMI_Order",
    when(col("BMI_Category") == "Niedowaga", 1)
    .when(col("BMI_Category") == "Waga prawidłowa", 2)
    .when(col("BMI_Category") == "Nadwaga", 3)
    .when(col("BMI_Category") == "Otyłość I stopnia", 4)
    .when(col("BMI_Category") == "Otyłość II stopnia", 5)
    .otherwise(6))  # Otyłość III stopnia)

bmi_distribution = bmi_distribution.orderBy("BMI_Order")


display(bmi_distribution)

BMI_Category,Count,Percentage,BMI_Order
Niedowaga,401,1.47,1
Waga prawidłowa,6307,23.05,2
Nadwaga,9990,36.5,3
Otyłość I stopnia,6160,22.51,4
Otyłość II stopnia,2698,9.86,5
Otyłość III stopnia,1811,6.62,6


Databricks visualization. Run in Databricks to view.

##### Wnioski:
- ##### Osoby z wagą prawidłową i niedowagą stanowią mały odsetek wśród chorych na serce, niecałe 25%
- ##### Większość przypadków chorób serca dotyczy osób z nadwagą i otyłością, ponad 75%

#### 6) Wykres słupkowy: Procentowy podział liczby osób z chorobą serca na palących i niepalących

In [ ]:
heart_disease_df = df.filter(col("HeartDisease") == "Yes")

smoking_distribution = heart_disease_df.groupBy("Smoking").agg(count("*").alias("Count"))


total_count = heart_disease_df.count()
smoking_distribution = smoking_distribution.withColumn(
    "Percentage", round((col("Count") / total_count) * 100, 2))


display(smoking_distribution)

Smoking,Count,Percentage
No,11334,41.41
Yes,16033,58.59


Databricks visualization. Run in Databricks to view.

##### Wnioski:
- ##### Palacze stanowią większość wśród osób z chorobą serca, ponad 58.5% co wskazuje na powiązanie pomiędzy występowaniem chorób serca a aktywnym paleniem papierosów.
- ##### Osoby niepalące również chorują na serce ale w miejszym stopniu, poniżej 41.5%

## Podsumowanie i wnioski

#### Styl życia odgrywa kluczową rolę w prewencji potencjalnych chorób serca. Ograniczenie palenia, regularna aktywność fizyczna oraz utrzymanie prawidłowej masy ciała mogą znacząco zmniejszyć ryzyko tych schorzeń. Wraz z rosnącym wiekiem organizm staje się bardziej podatny na choroby serca, dlatego profilaktyka staje się kluczowa.
#### Osoby z nadwagą i otyłością stanowią ponad 75% chorych na serce, co wskazuje na silny związek między BMI a ryzykiem wystąpienia choroby sercowo-naczyniowej. Palenie również okazuje się istotnym czynnikiem ryzyka, ponieważ palacze dominują wśród chorych (ponad 58%), podkreślając negatywny wpływ tego nawyku na serce.
#### Dodatkowo, mężczyźni częściej chorują na serce niż kobiety, mimo że rozkład płci w całej populacji jest zbliżony. Ryzyko wzrasta wraz z wiekiem, co wskazuje na potrzebę częstszych badań profilaktycznych, szczególnie u osób starszych.
#### Podsumowując, utrzymanie prawidłowej masy ciała, unikanie palenia oraz regularna aktywność fizyczna to kluczowe elementy, które mogą znacząco zmniejszyć ryzyko chorób serca i poprawić jakość życia.
